In [6]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


# New section

# New section

In [7]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [8]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

start_time = time.time()

spark.sql("""select date, count(*) from home_sales group by 1""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))


+----------+--------+
|date      |count(1)|
+----------+--------+
|2019-08-23|24      |
|2020-04-13|29      |
|2021-11-03|21      |
|2019-08-08|27      |
|2020-02-26|27      |
|2019-08-22|35      |
|2021-12-23|21      |
|2020-06-24|29      |
|2019-04-25|27      |
|2019-09-29|31      |
|2019-08-31|25      |
|2020-06-08|43      |
|2020-09-12|31      |
|2020-11-12|22      |
|2021-04-06|27      |
|2022-05-17|22      |
|2022-03-30|27      |
|2019-01-17|28      |
|2021-10-25|31      |
|2021-11-15|31      |
+----------+--------+
only showing top 20 rows

--- 4.194031238555908 seconds ---


In [12]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
start_time = time.time()

spark.sql("""select extract(YEAR from date) as Year,round(avg(price),2) as AveragePrice from home_sales where bedrooms==4 group by 1 order by 1""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|    300263.7|
|2020|   298353.78|
|2021|   301819.44|
|2022|   296363.88|
+----+------------+

--- 1.9535329341888428 seconds ---


In [13]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
start_time = time.time()

spark.sql("""select extract(YEAR from date) as Year,round(avg(price),2) as AveragePrice from home_sales where bedrooms==3 and bathrooms==3 group by 1 order by 1""").show()

print("--- %s seconds ---" % (time.time() - start_time))



+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|   287287.82|
|2020|   294204.16|
|2021|   294211.46|
|2022|   292725.69|
+----+------------+

--- 1.9539568424224854 seconds ---


In [14]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
start_time = time.time()

spark.sql("""select extract(YEAR from date) as Year,round(avg(price),2) as AveragePrice from home_sales where bedrooms==3 and bathrooms==3 and floors==2 and round(sqft_living,2)>=2000.00
group by 1 order by 1""").show()

print("--- %s seconds ---" % (time.time() - start_time))



+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|   289859.14|
|2020|   292289.09|
|2021|   296330.96|
|2022|   290242.99|
+----+------------+

--- 1.1082918643951416 seconds ---


In [16]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
    SELECT ROUND(AVG(view), 2) AS average_view
    FROM home_sales 
    WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.8057382106781006 seconds ---


In [17]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [18]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [19]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
    SELECT ROUND(AVG(view), 2) AS average_view
    FROM home_sales 
    WHERE price >= 350000
""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.3456082344055176 seconds ---


In [20]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("home_date_built_partitioned")


In [21]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('home_date_built_partitioned')

In [22]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_date_built_p')

In [23]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
    SELECT ROUND(AVG(view), 2) AS average_view
    FROM home_sales 
    WHERE price >= 350000
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.5168795585632324 seconds ---


In [24]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [25]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False